In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
data = pd.read_excel('EDGE_data.xlsx')

# Projection range
proj_years = np.array(list(range(2020, 2101, 5)))

In [4]:
# Timestep function
f_1 = lambda f_0, I_1, I_0, beta, D_1, D_0 : f_0*((I_1/I_0)**beta)*((D_1/D_0)**-0.03)

In [5]:
gdp_projection = data[(data['variable'] == 'GDP') & (data['year'] >= 2020)]['value'].tolist()
pop_projection = data[(data['variable'] == 'POP') & (data['year'] >= 2020)]['value'].tolist()

# Extract income projection from data
income_projection = data[(data['variable'] == 'GDP per capita') & (data['source'] == 'derived from SSP database') & (data['year'] >= 2020)]['value'].tolist()

# Income start point
income_2020 = income_projection[0]

# Extract density projection
density_projection = data[(data['variable'] == 'POP density') & (data['source'] == 'derived from SSP database') & (data['year'] >= 2020)]['value'].tolist()

# Density start
density_2020 = density_projection[0]

# Elasticities evolve over time horizon
income_elasticities = np.linspace(0.42, 0.294, num = len(proj_years))

# Create empty array for demand values
floor_demands =  np.zeros(len(proj_years))

In [6]:
len(density_projection) == len(floor_demands)

True

# Floor Space Demand

In [ ]:
# Calibrate using parameters from Table SI4 (found by regression)
floor_demands[0] = 0.61*(income_2020**0.42)*(density_2020**-0.03)

In [ ]:
# Calculate demands
for index in range(1,len(proj_years)):
    floor_demands[index] = f_1(floor_demands[index-1], income_projection[index], income_projection[index-1], income_elasticities[index], density_projection[index], density_projection[index-1])

In [ ]:
# Ratio of com to res given as 0.35 (for income levels above 20k, which all in projection are)
res_floor_demands = floor_demands
com_floor_demands = res_floor_demands * 0.35

In [ ]:
# Plotting data
plt.plot(proj_years, res_floor_demands, color = 'red', label = 'Residential')
plt.plot(proj_years, com_floor_demands, color = 'blue', label = 'Commercial')
plt.title('EDGE Projection of Floor Space Demand (Canada)')
plt.xlabel('Year')
plt.ylabel('Floor Space Demand (people/sq. km)')
plt.legend()
plt.show()

In [ ]:
# Save results and input for testing other functions
np.save('com_floor_demands', com_floor_demands)
np.save('res_floor_demands', res_floor_demands)
np.save('income_projection', income_projection)
np.save('density_projection', density_projection)

# Space heating/cooling demand

In [ ]:
# HDD and CDD estimates from figure in EDGE source

hdd_projection = np.linspace(1300, 725, num = len(proj_years))
cdd_projection = np.linspace(1250, 1800, num = len(proj_years))
gamma = np.linspace(1, 0.7, num = len(proj_years))

In [ ]:
SH = lambda F, y, hdd, cdd: F*y*(0.45 + np.exp(1.4 + (-5e-4)*(hdd+cdd)))*0.07*hdd
space_heating = np.zeros(len(proj_years))

for index in range(0, len(proj_years)):
    space_heating[index] = SH(floor_demands[index], gamma[index], hdd_projection[index], cdd_projection[index])
    

In [ ]:
plt.plot(proj_years, space_heating)
plt.title('Space heating demand in EDGE')
plt.show()

In [ ]:
SC = lambda F, y, hdd, cdd, I : F*y*(0.45 + np.exp(1.4 + (-5e-4)*(hdd+cdd)))*cdd*(1-(0.949*np.exp(-0.0018*cdd)))*(0.16/(1 + np.exp((20950 - I)/3774)))
space_cooling = np.zeros(len(proj_years))

for index in range (0, len(proj_years)):
    space_cooling[index] = SC(floor_demands[index], gamma[index], hdd_projection[index], cdd_projection[index], income_projection[index])

In [ ]:
plt.plot(proj_years, space_cooling)
plt.title('Space cooling demand in EDGE')

# Appliances and Lighting

In [ ]:
AL = lambda I : np.exp(-7 + 0.3*(np.log(I) + (1/np.sqrt(I))))
appliances = np.zeros(len(proj_years))

for index in range (0, len(proj_years)):
    appliances[index] = AL(income_projection[index])

# Convert to MJ
appliances = appliances * 1e6

In [ ]:
plt.plot(proj_years, appliances)
plt.title('Appliances and lighting demand in EDGE')

# Water heating

In [ ]:
phi_1 = np.linspace(8.19, 8.19*0.6, num = len(proj_years))

WH = lambda phi_1, I : phi_1*(1 + np.exp((24258 - I)/8622))
water_heating = np.zeros(len(proj_years))

for index in range(0, len(proj_years)):
    water_heating[index] = WH(phi_1[index], income_projection[index])

# Convert to MJ
water_heating = water_heating * 1e3

In [ ]:
plt.plot(proj_years, water_heating)
plt.title('Water heating demand in EDGE')

# Cooking

In [ ]:
cooking = np.empty(len(proj_years))
cooking.fill(1.76)

# Convert to MJ
cooking = cooking * 1e3

# Aggregate all demands

In [ ]:
usefuls = {'Space Heating': space_heating, 'Space Cooling': space_cooling, 'Appliances and Lighting': appliances, 'Water Heating': water_heating, 'Cooking': cooking}
useful_energy = space_heating + space_cooling + water_heating + appliances

In [ ]:
plt.plot(proj_years, useful_energy)
plt.title('Useful energy demand in EDGE')

In [ ]:
plt.figure(figsize = (20,20))
plt.rc('font', size = 25)
plt.plot(gdp_projection, useful_energy)
ax = plt.gca()
plt.scatter(gdp_projection, useful_energy)
for i, label in enumerate(proj_years):
    plt.annotate(label, (gdp_projection[i], useful_energy[i]))
plt.ylabel('Useful energy (MJ/cap)')
plt.xlabel('GDP (billion 2005 USD)')
plt.title('EDGE useful energy demand (Canada)')
plt.savefig('edge_useful.jpg')
plt.show()

# Final to Useful Conversion

In [ ]:
# Conversion efficiencies
elec_conv = 


In [ ]:
sh_wh_carriers = {'Electricity': 0.4, 'Other Grids': 0.5, 'Liquids/Modern Biomass': 0.1}
sc_carriers = {'Electricity': 0.9, 'Other Grids': 0.1}
app_carriers = {'Electricity': 1}

FE = lambda ue, eff, share : (ue/np.sum(eff*share))

FE_sh = np.zeros(len(proj_years))

# for index in range(0, len(proj_years)):
#     FE_sh[index] = FE(space_heating
